In [ ]:
# Пример выбора записи из музыкальной БД.

In [75]:
import csv
import json
import random
import re
from typing import Dict, List, Optional
import requests
import time
import sys

class audioplay:
    def __init__(self, host: str = "http://127.0.0.1:5000", csv_file: str = "music_base.csv"):
        """
        Инициализация класса. Считывает файл music_base.csv
        """
        self.host = host
        self.music_base = []
        try:
            with open(csv_file, 'r', encoding='utf-8-sig') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.music_base.append(row)
            print(f"Загружено {len(self.music_base)} записей из {csv_file}")
        except FileNotFoundError:
            print(f"Файл {csv_file} не найден. База данных пуста.")
        except Exception as e:
            print(f"Ошибка при чтении файла {csv_file}: {e}")
############################

    def find_in_base(self, artist: str = "", title: str = "", album: str = "", genre: str = "") -> Dict:
        """
        Поиск наиболее подходящей записи в базе данных
        
        Args:
            artist: Имя исполнителя для поиска
            title: Название трека для поиска
            album: Название альбома для поиска
            genre: Жанр для поиска
        
        Returns:
            Dict: Наиболее подходящая запись в виде словаря или False, если не найдено
        """
        if not self.music_base:
            return False
        
        # Создаем список критериев поиска (не пустых)
        search_criteria = []
        if artist:
            search_criteria.append(('Artist', artist))
        if title:
            search_criteria.append(('Title', title))
        if album:
            search_criteria.append(('Album', album))
        if genre:
            search_criteria.append(('Genre', genre))
        
        # Если нет критериев поиска, возвращаем False
        if not search_criteria:
            return False
        
        best_matches = []
        best_score = 0
        
        for record in self.music_base:
            score = 0
            for field, value in search_criteria:
                # Сравниваем без учета регистра
                if record.get(field, '').lower() == value.lower():
                    score += 1
            
            if score > 0:  # Только записи с хотя бы одним совпадением
                if score > best_score:
                    best_score = score
                    best_matches = [record]
                elif score == best_score:
                    best_matches.append(record)
        
        # Если найдены совпадения, выбираем случайное из лучших
        if best_matches:
            return random.choice(best_matches)
        else:
            # Если критерии есть, но нет совпадений, возвращаем False
            return False

###################
    
    def audio_play(self, playfile: Optional[Dict] = None):
        """
        Функция воспроизведения аудио
        
        Args:
            host: Хост для воспроизведения
            playfile: Словарь с данными о треке
        """
        if playfile:
            print(playfile)
            print(f"SourceFile = {playfile.get('SourceFile', 'N/A')}")
            SourceFile = playfile.get('SourceFile', 'N/A')
            print(f"FileName = {playfile.get('FileName', 'N/A')}")
            print(f"Artist = {playfile.get('Artist', 'N/A')}")
            print(f"Album = {playfile.get('Album', 'N/A')}")
            print(f"Title = {playfile.get('Title', 'N/A')}")
            print(f"Genre = {playfile.get('Genre', 'N/A')}")
            print(f"Воспроизведение на хосте (адрес:порт): {self.host}")
            ##########
            try:                
                
                resp = requests.post(
                    f"{self.host}/play",
                    json={"file_path": SourceFile},
                    headers={"Content-Type": "application/json"}
                )
                print(f"✅ Сервис доступен: {resp.json()}")
                return resp
            except:
                print("❌ Сервис не доступен!")
                print("Возможно надо запустить сначала сервис: python audio_player_service.py")
                return False
        else:
            print("Нет данных для воспроизведения")
            return False
    
    def audio_stop(self):
        """Функция остановки воспроизведения"""
        print("Play stopping")
        try:
            resp = requests.post(f"{self.host}/stop")
        except Exception as e:
            print("❌ Сервис не доступен!")
            print(e)
            return False
        return resp
        
    
    def audio_status(self):
        """Функция получения статуса воспроизведения"""
        print("Status of playing")
        try:
            resp = requests.get(f"{self.host}/status")
        except:
            print("❌ Сервис не доступен!")
            return False
        return resp
        
    
    def parse_xml_arguments(self, xml_content: str) -> Dict:
        """
        Парсинг аргументов из XML-строки
        
        Args:
            xml_content: XML-строка с аргументами
        
        Returns:
            Dict: Словарь с аргументами
        """
        args = {}
        
        # Извлекаем значения аргументов с помощью регулярных выражений
        artist_match = re.search(r'<artist>(.*?)</artist>', xml_content, re.DOTALL)
        title_match = re.search(r'<title>(.*?)</title>', xml_content, re.DOTALL)
        album_match = re.search(r'<album>(.*?)</album>', xml_content, re.DOTALL)
        genre_match = re.search(r'<genre>(.*?)</genre>', xml_content, re.DOTALL)
        
        if artist_match:
            args['artist'] = artist_match.group(1).strip()
        if title_match:
            args['title'] = title_match.group(1).strip()
        if album_match:
            args['album'] = album_match.group(1).strip()
        if genre_match:
            args['genre'] = genre_match.group(1).strip()
        
        return args
    
    def tool_call(self, xml_string: str) -> Dict:
        """
        Обработка XML-запроса
        
        Args:
            xml_string: XML-строка с вызовом инструмента
        
        Returns:
            Dict: Результат выполнения в формате JSON
        """
        # Удаляем все после </tool_call>
        if '</tool_call>' in xml_string:
            xml_string = xml_string.split('</tool_call>')[0] + '</tool_call>'
        
        # Извлекаем имя функции из XML
        name_match = re.search(r'<name>(.*?)</name>', xml_string)
        
        if not name_match:
            return {
                "status": "error",
                "message": "Invalid XML format: missing <name> tag"
            }
        
        function_name = name_match.group(1)
        
        # Проверяем, что вызов начинается с audioplay.
        if not function_name.startswith('audioplay.'):
            return {
                "status": "error",
                "message": "Don't run non audioplay tool."
            }
        
        # Извлекаем название метода
        method_name = function_name.split('.')[1] if '.' in function_name else ''
        
        # Обработка различных методов
        if method_name == 'play_request':
            # Парсим аргументы
            args = self.parse_xml_arguments(xml_string)
            
            # Ищем подходящую запись в базе
            found_record = self.find_in_base(
                artist=args.get('artist', ''),
                title=args.get('title', ''),
                album=args.get('album', ''),
                genre=args.get('genre', '')
            )
            
            if found_record:
                # Вызываем функцию воспроизведения
                if self.audio_play(playfile=found_record):
                    # Возвращаем найденную запись
                    return {
                        "status": "success",
                        "data": found_record
                    }
                else: # Запустить не удалось
                    return {
                        "status": "error",
                        "message": "Can't start playing"
                    }
            else: # Нет подходящей записи
                return {
                    "status": "error",
                    "message": "No matching records found"
                }
        
        elif method_name == 'play_stop':
            self.audio_stop()
            return {
                "status": "success",
                "message": "Playback stopped"
            }
        
        elif method_name == 'play_status':
            self.audio_status()
            return {
                "status": "success",
                "message": "Status requested"
            }
        
        else:
            return {
                "status": "error",
                "message": "Unknown audioplay method"
            }


In [76]:
player = audioplay(host="http://127.0.0.1:5000", csv_file = "music_base.csv")

Загружено 812 записей из music_base.csv


In [77]:

# Примеры вызовов
test_calls = [
    # Корректный запрос на воспроизведение
    '<tool_call><name>audioplay.play_request</name><arguments><artist>John Legend</artist><title>All of Me</title><genre>Pop</genre><album>Love in the Future</album></arguments></tool_call>',
    
    # Запрос на воспроизведение с тегом <end_of_turn>
    '<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title>What a Wonderful World</title><genre>Jazz</genre><album></album></arguments></tool_call><end_of_turn>',
    
    # Еще один
    '<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title>Come sunday</title><genre>Jazz</genre><album></album></arguments></tool_call><end_of_turn>',
    
    
    # Запрос на остановку
    '<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>',
    
    # Запрос статуса
    '<tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>',
    
    # Неверное имя инструмента
    '<tool_call><name>othertool.play_request</name><arguments></arguments></tool_call>',
    
    # Неизвестный метод audioplay
    '<tool_call><name>audioplay.unknown_method</name><arguments></arguments></tool_call>'
]

for i, test_call in enumerate(test_calls):
    print(f"\n{'='*50}")
    print(f"Тест #{i+1}:")
    print(f"Входные данные: {test_call[:150]}...")
    result = player.tool_call(test_call)
    print(f"Результат: {json.dumps(result, indent=2)}")


Тест #1:
Входные данные: <tool_call><name>audioplay.play_request</name><arguments><artist>John Legend</artist><title>All of Me</title><genre>Pop</genre><album>Love in the Futu...
Результат: {
  "status": "error",
  "message": "No matching records found"
}

Тест #2:
Входные данные: <tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title>What a Wonderful World</title><genre>Jazz</genre><albu...
{'SourceFile': '/home/m1/music1/Duke Ellington - Victor recordings 1927-1931/008. Blue Bubbles.mp3', 'FileName': '008. Blue Bubbles.mp3', 'Artist': 'Duke Ellington and his Orchestra', 'Album': 'Victor', 'Title': 'Blue Bubbles', 'Genre': 'Jazz'}
SourceFile = /home/m1/music1/Duke Ellington - Victor recordings 1927-1931/008. Blue Bubbles.mp3
FileName = 008. Blue Bubbles.mp3
Artist = Duke Ellington and his Orchestra
Album = Victor
Title = Blue Bubbles
Genre = Jazz
Воспроизведение на хосте (адрес:порт): http://127.0.0.1:5000
✅ Сервис доступен: {'file': '/home/m1

In [68]:
player = audioplay("music_base.csv")

test_call = "<tool_call><name>audioplay.play_request</name><arguments><artist>Louis Armstrong</artist><title>Come sunday</title><genre>Jazz</genre><album></album></arguments></tool_call><end_of_turn>"
result = player.tool_call(test_call)
print(f"result = {result}")

Загружено 812 записей из music_base.csv
{'SourceFile': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'FileName': 'Duke Ellington - Come sunday.mp3', 'Artist': 'Duke Ellington', 'Album': 'JazzmastersMPEG', 'Title': 'Come sunday', 'Genre': 'Jazz'}
SourceFile = /home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3
FileName = Duke Ellington - Come sunday.mp3
Artist = Duke Ellington
Album = JazzmastersMPEG
Title = Come sunday
Genre = Jazz
Воспроизведение на хосте (адрес:порт): http://127.0.0.1:5000
✅ Сервис доступен: {'file': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'message': 'Начато воспроизведение: Duke Ellington - Come sunday.mp3', 'status': 'success'}
result = {'status': 'success', 'data': {'SourceFile': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'FileName': 'Duke Ellington - Come sunday.mp3', 'Artist': 'Duke Ellington', 'Album': 'Jazz

In [47]:
!curl -X POST http://localhost:5000/stop

{"message":"\u0412\u043e\u0441\u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u043e\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u0435\u043d\u043e","status":"success"}


In [46]:
BASE_URL = "http://127.0.0.1:5000"
test_file = "/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3"
resp = requests.post(
                    f"{BASE_URL}/play",
                    json={"file_path": test_file},
                    headers={"Content-Type": "application/json"}
                )

In [58]:
resp = requests.post(f"{BASE_URL}/stop")

In [69]:
test_call = '<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>'
result = player.tool_call(test_call)

Play stopping
❌ Сервис не доступен!
name 'host' is not defined
